In [23]:
from pathlib import Path
import os
import SimpleITK as sitk
import numpy as np
import csv
import glob
import pandas as pd
try:
    from tqdm import tqdm  # long waits are not fun
except:
    print('TQDM does make much nicer wait bars...')
    tqdm = lambda x: x

In [24]:
resampling_paths = "../../luna16/LUNA16_resampling/"

## Nội suy mode="nearest" thây đổi kích thước của spacing

In [28]:
import numpy as np
from scipy import ndimage

def rescale_spacing(image, old_spacing, new_spacing):
    # Tính toán tỷ lệ thay đổi spacing
    scale_factor = np.array(old_spacing) / np.array(new_spacing)

    # Tính toán kích thước mới của ảnh CT
    new_shape = np.round(image.shape * scale_factor)
    new_shape = new_shape.astype(int)

    # Thay đổi kích thước ảnh CT bằng phương pháp nội suy
    resized_image = ndimage.zoom(image, scale_factor, mode='nearest')

    return resized_image

# Quá trình resampling thành spacing = [1, 1, 1]
for subset in range(0, 1):
    # break

    save_path = f"../../luna16/LUNA16_resampling/subset{subset}"

    if not os.path.exists(save_path):
        os.makedirs(save_path)

    luna_paths = glob.glob(f"../../luna16/LUNA16/subset{subset}/*.mhd")
    new_spacing = np.array([1.0, 1.0, 1.0])

    for path in tqdm(luna_paths):
        uid = path.split("\\")[-1][:-4]
        ct_mhd = sitk.ReadImage(path)
        ct_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)

        old_spacing = np.array(ct_mhd.GetSpacing()[::-1])
        origin = np.array(ct_mhd.GetOrigin())
        direction = np.array(ct_mhd.GetDirection())

        
        resized_image = rescale_spacing(ct_a, old_spacing, new_spacing)

        np.save(f"{save_path}/{uid}_resampling.npy", resized_image)
        np.save(f"{save_path}/{uid}_origin.npy", origin)
        np.save(f"{save_path}/{uid}_direction", direction)


100%|██████████| 89/89 [36:04<00:00, 24.32s/it]


#### Test

In [27]:
test_uid = "1.3.6.1.4.1.14519.5.2.1.6279.6001.287966244644280690737019247886"

luna_path = glob.glob(f"../../luna16/LUNA16/subset1/{test_uid}.mhd")[0]

ct_mhd = sitk.ReadImage(luna_path)
ct_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)

print(ct_a.shape)
print(ct_mhd.GetSpacing()[::-1])

resampling_path = glob.glob(f"../../luna16/LUNA16_resampling/subset1/{test_uid}_resampling.npy")[0]

img = np.load(resampling_path)

print(img.shape)


(123, 512, 512)
(2.5, 0.8203120231628418, 0.8203120231628418)
(308, 420, 420)


In [10]:
# # Kiểm tra phân phối số lượng lát cắt trong ảnh CT
# paths = glob.glob(f"{resampling_paths}/subset*/*_resampling.npy")
# shapes = []

# for path in tqdm(paths):
#     image = np.load(path)
#     shapes.append(image.shape)


100%|██████████| 888/888 [32:54<00:00,  2.22s/it]
